#### 引入需要的库

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import pdb

#### 将12个月的csv数据合并为一个文件

In [ ]:
#pandas 读取 csv 文件的方法 read_csv
#file = pd.read_csv('./Sales_Data/Sales_April_2019.csv')
fn :str = ''
path :str = './Sales_Data/'
all_data = pd.DataFrame()
for fn in os.listdir(path):
    if fn.endswith('.csv'):
        fn = os.path.join(path,fn) # 获得文件完整路径
        all_data = pd.concat([all_data,pd.read_csv(fn)])
        # 注意 ⚠️ 通过 concat 之后得到的文件，index 是乱的，必须使用 reset_index() 重建 index
print(all_data.shape)

#### 问题1: 哪个月份销售额最高，销售额是多少？

In [ ]:
#all_data.head()

从 Order Date 中分离出月份

In [ ]:
# 本cell 用于试验，在删除特定行数据的实现上，以下两种方法输出的结果可能会造成不一样，因为index的缘故。
all_data = all_data.reset_index(drop=True)
# 关键就在上面这一句，因为 all_data 以前是通过多个 csv文件concat出来的，index有重复，需要重建index 才行。
print('all_data.shape = ')
print(all_data.shape)

# 方案1: 直接将符合条件的行保留
all_data1 = all_data[all_data['Order Date']!='Order Date']

print('\nall_data1.shape = ')
print(all_data1.shape)

# 方案2: 获得不符合条件的行，drop掉
# 注意：all_data 是通过多个 csv文件 concat出来的，不重建index的话，
# 这里得到的index实际数据中可能会出现多个，所以drop掉355个index，实际上将删除更多的行。
uselessrows = all_data[all_data['Order Date']=='Order Date'].index

print('\nlen of uselessrows = ')
print(len(uselessrows))

all_data2 = all_data.drop(uselessrows)

print('\nall_data2.shape = ')
print(all_data2.shape)



In [ ]:
# 将 Order Date 列转为 Date 类型
# 执行all_data['Order Date'] = pd.to_datetime(all_data['Order Date']) 失败
# 发现有字符串‘Order Date’，应该是合并数据时的表头行，找出来删掉
all_data = all_data[all_data['Order Date']!='Order Date']
# 此时应该删掉了
all_data['Order Date'] = pd.to_datetime(all_data['Order Date']) 
# all_data.head()
# 新建一列 月份
all_data['Month'] = all_data['Order Date'].dt.month #为何返回的是浮点数？
# 将浮点数转为整形，执行 all_data['Month'] = all_data['Month'].astype('int32') 失败，提示有NA数据
# 尝试找出有NA数据的行
#uselessrows = all_data[all_data.isna()]
#uselessrows.head()
all_data = all_data.dropna(how='all') # 这里参数how='all'，只有所有列都是Na，才能删除。
# 此时应该已经删除Na数据，重新执行月份转整形操作
all_data['Month'] = all_data['Month'].astype('int32')
#all_data.head()
# 成功！
print(all_data.shape)

按照月份汇总

In [ ]:
# 数据中销售数量和单价都是字符，需要先转成数字
all_data['Quantity Ordered'] = pd.to_numeric(all_data['Quantity Ordered'])
all_data['Price Each'] = pd.to_numeric(all_data['Price Each'])

all_data['Sales'] = all_data.apply(lambda x:x['Quantity Ordered']*x['Price Each'],axis=1)
grouped = all_data.groupby('Month')
sales = grouped['Sales'].sum()
months = range(1,13)
plt.bar(months,sales)
plt.xticks(months)
plt.xlabel('Month')
plt.ylabel('Sales')
plt.show()

#### 问题2：哪个城市的销售额最高？

In [ ]:
#all_data.head()

思路就是从 Purchase Address 中分离出不重复的 City，在 groupby， 取 Sales 合计，画直方图

In [ ]:
# 从 Purchase Address 中分离出不重复的 city
# 测试
#all_data['Purchase Address'].apply(lambda x:x.split(', ')[1] + ', ' +x.split(', ')[2].split(' ')[0]).head()
# 测试通过，增加新列
all_data['City'] = all_data['Purchase Address'].apply(lambda x:x.split(', ')[1] + ', ' +x.split(', ')[2].split(' ')[0])
# groupby, sum
grouped_by_city = all_data[['Sales','City']].groupby('City')
cities = [a for a,b in grouped_by_city]  
# 上面循环中之所以是 a for a,b ，是因为 grouped_by_city 是由键值对儿组成的
#for key, values in grouped_by_city:
#    print(values, "\n\n")

sales_city = grouped_by_city['Sales'].sum()
plt.bar(cities,sales_city)
plt.xticks(rotation='vertical')
plt.xlabel('City name')
plt.ylabel('Sales in USD')
plt.show()

#### 问题3: 什么时间段应该打广告？

思路：根据下单时间，根据几点钟汇总出订单数量，画曲线图

In [ ]:
all_data.head()

In [ ]:
# 增加出“小时” 列
# 测试
# print(all_data['Order Date'].dt.hour)
all_data['Hour'] = all_data['Order Date'].dt.hour
all_data.head()

In [ ]:
# 根据 Hour 进行 groupby， 对 Quantity Ordered 求和
grouped_by_hour = all_data[['Quantity Ordered','Hour']].groupby('Hour')
num_ordered = grouped_by_hour['Quantity Ordered'].sum()
hours = [hour for hour,x in grouped_by_hour]
plt.plot(hours,num_ordered,'r-')
plt.xticks(hours)
plt.xlabel('Hour')
plt.ylabel('Quantity Ordered')
plt.grid(axis='y')
plt.show()

#### 问题4：哪些产品经常一起卖出？

In [ ]:
all_data.head()

思路：如果是一起卖出的，那么订单号就是相同的，

In [ ]:
# 先把 Order ID 重复的列出来看看
#pd.concat(g for _, g in all_data[['Order ID','Product']].groupby("Order ID") if len(g) > 1)
# 上面这句话执行太慢了，换个思路

In [ ]:
thedata = all_data.copy() # 这里不想修改 all_data，所以复制了一下
thedata = thedata[thedata['Order ID'].duplicated(keep=False)] # 只保留订单号重复的数据
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.duplicated.html
# keep{‘first’, ‘last’, False}, default ‘first’
#    Determines which duplicates (if any) to mark.
#     first : 重复的数据中，第一个为 False，其余为 True
#     last : 重复的数据中，最后一个为 False，其余为 True
#     False : 重复的数据中，全部设为 True
thedata['Grouped'] = thedata.groupby('Order ID')['Product'].transform(lambda x:','.join(x))
thedata = thedata.drop_duplicates(['Order ID','Grouped'])
thedata.head()

注意⚠️，这里不能直接写成 thedata = all_data , 因为 Python 中一切皆对象，直接赋值实际上是「传址」，
详见 https://www.zhihu.com/question/26614862

In [ ]:
# Referenced: https://stackoverflow.com/questions/52195887/counting-unique-pairs-of-numbers-into-a-python-dictionary
from itertools import combinations
from collections import Counter
count = Counter()
for row in thedata['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list,2)))
for key, value in count.most_common(10):
    print(key,value)

#### 问题5：什么产品卖的最好，为什么？

In [ ]:
all_data.head()

思路：按照产品groupby，然后对Quantity Ordered求和。
至于回答为什么，可以在同一个图里展示单价曲线图Price Each，看出单价和销量的关系。

In [ ]:
# groupby product 并对销量求和
grouped_product_quantity = all_data.groupby('Product')['Quantity Ordered'].sum()
products = [product for product,_ in all_data.groupby('Product')]
# groupby product 并对单价求平均值
each_price = all_data.groupby('Product')['Price Each'].mean()

fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.bar(products,grouped_product_quantity)
ax2.plot(products,each_price,'y-')
ax1.set_xlabel('Product')
ax1.set_xticklabels(products,rotation='vertical')
#ax1.grid(axis='y',color='w',linewidth=1,alpha=0.2)
ax1.set_axisbelow(True)
ax1.grid(b=None,axis='x')
ax2.grid()
plt.show()

#### End